In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import zipfile

In [6]:
earnings = pd.read_excel('data/earnings formatted.xlsx')
earnings = earnings[earnings['Date'] > dt.datetime(2020,1,1)]
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BRK'].index) # drop berkshire hathaway
earnings = earnings.drop(earnings[earnings['Ticker'] == 'BF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'UHALB'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'LGF'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'METCV'].index)
earnings = earnings.drop(earnings[earnings['Ticker'] == 'GTXI'].index)
earnings

,Unnamed: 0,Ticker,Quarter,Date,Time
0,0,MSFT,2024:Q4,2024-07-30,16:00
1,1,MSFT,2024:Q3,2024-04-25,16:00
2,2,MSFT,2024:Q2,2024-01-30,16:00
3,3,MSFT,2024:Q1,2023-10-24,16:04
4,4,MSFT,2023:Q4,2023-07-25,16:00
...,...,...,...,...,...
182047,182047,WLLBW,2020:Q4,2021-02-24,17:34
182048,182048,WLLBW,2020:Q3,2020-11-05,06:02
182049,182049,WLLBW,2020:Q2,2020-08-05,08:00
182050,182050,WLLBW,2020:Q1,2020-05-18,09:00


In [7]:
returns = pd.read_csv('returns.csv.zip',compression='zip')
returns = returns.drop(returns[returns['Ticker'] == 'BRK'].index)
returns = returns.drop(returns[returns['Ticker'] == 'BF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'UHALB'].index)
returns = returns.drop(returns[returns['Ticker'] == 'LGF'].index)
returns = returns.drop(returns[returns['Ticker'] == 'METCV'].index)
returns = returns.drop(returns[returns['Ticker'] == 'GTXI'].index)
returns

,Date,Ticker,Return
0,2020-01-03 00:00:00+00:00,A,-0.016056
1,2020-01-06 00:00:00+00:00,A,0.002956
2,2020-01-07 00:00:00+00:00,A,0.003065
3,2020-01-08 00:00:00+00:00,A,0.009873
4,2020-01-09 00:00:00+00:00,A,0.015712
...,...,...,...
2990267,2024-09-16 00:00:00+00:00,ZYXI,-0.022086
2990268,2024-09-17 00:00:00+00:00,ZYXI,0.027603
2990269,2024-09-18 00:00:00+00:00,ZYXI,-0.001221
2990270,2024-09-19 00:00:00+00:00,ZYXI,0.018337


In [10]:
#Disclaimer: idk if this code actually works, its based off sample.ipynb code - Andy
# Took 13:20 for this to finish running for me - Andy

ticker_list = returns['Ticker'].unique()

tickerdata_list = []

for ticker in ticker_list:
    ticker_data = yf.Ticker(ticker).history(start=dt.datetime(2020,1,1))
    ticker_data = ticker_data.reset_index()
    
    tickerdata_list.append(ticker_data)

y_finance = pd.concat(tickerdata_list, keys=ticker_list).reset_index()
y_finance = y_finance.rename(columns={'level_0':'Ticker'})
y_finance = y_finance.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'level_1'])
y_finance

In [24]:
y_finance['Date'] = pd.to_datetime(y_finance['Date'])
y_finance['Date'] = y_finance['Date'].dt.tz_localize(None)
returns['Date'] = pd.to_datetime(returns['Date'])
returns['Date'] = returns['Date'].dt.tz_localize(None)

merge_list = ['Date', 'Ticker']
merged_df = returns.merge(y_finance, on=merge_list)
merged_df.head()

C:\Users\landr\AppData\Local\Temp\ipykernel_11204\413031144.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  y_finance['Date'] = pd.to_datetime(y_finance['Date'])


AttributeError: Can only use .dt accessor with datetimelike values

In [14]:
merged_df['log 1d ret'] = np.log1p(merged_df['Return'])
merged_df['ret sum'] = merged_df['log 1d ret'].cumsum()

merged_df['ret sum 3 shift'] = merged_df['ret sum'].shift(-3)
merged_df['3d ret'] = np.exp(merged_df['ret sum 3 shift'] - merged_df['ret sum']) - 1

merged_df['ret sum 7 shift'] = merged_df['ret sum'].shift(-7)
merged_df['7d ret'] = np.exp(merged_df['ret sum 7 shift'] - merged_df['ret sum']) - 1
merged_df.head()

c:\Users\landr\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Date,Ticker,Return,Close,Volume,log 1d ret,ret sum,ret sum 3 shift,3d ret,ret sum 7 shift,7d ret
0,2020-01-03,A,-0.016056,83.438141,1118300,-0.016186,-0.016186,-0.000349,0.015963,0.023457,0.040440
1,2020-01-06,A,0.002956,83.684799,1993200,0.002952,-0.013234,0.015241,0.028884,0.030592,0.044801
2,2020-01-07,A,0.003065,83.941315,1684700,0.003061,-0.010174,0.018901,0.029501,0.040026,0.051481
3,2020-01-08,A,0.009873,84.770073,1847600,0.009825,-0.000349,0.017416,0.017924,0.047376,0.048883
4,2020-01-09,A,0.015712,86.101990,1912700,0.015590,0.015241,0.023457,0.008250,0.041701,0.026814


In [16]:
earnings['Date'] = pd.to_datetime(earnings['Date'])
earnings['Date'] = earnings['Date'].dt.tz_localize(None)
final_df = earnings.merge(merged_df, on=merge_list)
final_df

,Unnamed: 0,Ticker,Quarter,Date,Time,Return,Close,Volume,log 1d ret,ret sum,ret sum 3 shift,3d ret,ret sum 7 shift,7d ret
0,0,MSFT,2024:Q4,2024-07-30,16:00,-0.008928,422.159119,32687600,-0.008968,212.879873,212.845157,-0.034120,212.830856,-0.047834
1,1,MSFT,2024:Q3,2024-04-25,16:00,-0.024495,397.604889,40586500,-0.024800,212.821751,212.797117,-0.024333,212.857444,0.036337
2,2,MSFT,2024:Q2,2024-01-30,16:00,-0.002758,406.369080,33477600,-0.002762,212.845402,212.851818,0.006437,212.858821,0.013510
3,3,MSFT,2024:Q1,2023-10-24,16:04,0.003674,328.067505,31153600,0.003667,212.633387,212.631206,-0.002178,212.685811,0.053823
4,4,MSFT,2023:Q4,2023-07-25,16:00,0.017009,347.629150,41637700,0.016866,212.693419,212.656829,-0.035928,212.621609,-0.069292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47437,182044,WLLBW,2021:Q3,2021-11-03,17:24,-0.049393,10.970000,1800,-0.050655,263.118734,263.165047,0.047402,263.126000,0.007293
47438,182045,WLLBW,2021:Q2,2021-08-03,16:30,-0.028571,5.100000,8300,-0.028988,262.352810,262.446336,0.098039,262.428318,0.078431
47439,182046,WLLBW,2021:Q1,2021-05-03,17:44,0.067742,3.310000,3700,0.065546,261.920518,261.870972,-0.048338,261.822182,-0.093656
47440,182047,WLLBW,2020:Q4,2021-02-24,17:34,-0.024055,2.840000,37900,-0.024349,261.767374,261.753189,-0.014084,261.705648,-0.059859


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(final_df['Volume'], final_df[''])

In [19]:


y_finance

,Ticker,Date,Close,Volume
0,MSFT,2020-01-02 00:00:00-05:00,153.938187,22622100
1,MSFT,2020-01-03 00:00:00-05:00,152.021454,21116200
2,MSFT,2020-01-06 00:00:00-05:00,152.414352,20813700
3,MSFT,2020-01-07 00:00:00-05:00,151.024689,21634100
4,MSFT,2020-01-08 00:00:00-05:00,153.430252,27746500
...,...,...,...,...
2984355,WLLBW,2024-09-17 00:00:00-04:00,10.510000,2500
2984356,WLLBW,2024-09-18 00:00:00-04:00,10.710000,7200
2984357,WLLBW,2024-09-19 00:00:00-04:00,12.000000,6800
2984358,WLLBW,2024-09-20 00:00:00-04:00,12.000000,14300
